In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from paths import paths

In [2]:
df = pd.read_csv(paths.TRAIN_CSV)
label_cols = df.columns[-6:]
number_experts = df[label_cols].sum(axis = 1)
for label in label_cols:
    df[label] = df[label] / number_experts
display(df)

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0.0,0.0,0.0,1.0,0.0,0.0


In [3]:
dataframe = df[['eeg_id', 'eeg_sub_id', 'eeg_label_offset_seconds', 'seizure_vote']]
print(dataframe.shape)
dataframe.head()

(106800, 4)


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,seizure_vote
0,1628180742,0,0.0,1.0
1,1628180742,1,6.0,1.0
2,1628180742,2,8.0,1.0
3,1628180742,3,18.0,1.0
4,1628180742,4,24.0,1.0


In [4]:
amc_matrix = np.zeros((dataframe.shape[0], 61))
for i, idx in enumerate(tqdm(dataframe.index)):
    row = dataframe.loc[idx]
    id = int(row.eeg_id)
    sub_id = int(row.eeg_sub_id)
    offset = row.eeg_label_offset_seconds
    amc_matrix[i,:] = np.load(paths.HJORTH_10 + f'{id}_{sub_id}.npy')

  0%|          | 0/106800 [00:00<?, ?it/s]

In [5]:
columns = []
for name in paths.EEG_LABELS:
    columns.append(name + '_activity')
    columns.append(name + '_mobility')
    columns.append(name + '_complexity')
columns.append('generalized')
print(columns)

['Fp1_activity', 'Fp1_mobility', 'Fp1_complexity', 'F3_activity', 'F3_mobility', 'F3_complexity', 'C3_activity', 'C3_mobility', 'C3_complexity', 'P3_activity', 'P3_mobility', 'P3_complexity', 'F7_activity', 'F7_mobility', 'F7_complexity', 'T3_activity', 'T3_mobility', 'T3_complexity', 'T5_activity', 'T5_mobility', 'T5_complexity', 'O1_activity', 'O1_mobility', 'O1_complexity', 'Fz_activity', 'Fz_mobility', 'Fz_complexity', 'Cz_activity', 'Cz_mobility', 'Cz_complexity', 'Pz_activity', 'Pz_mobility', 'Pz_complexity', 'Fp2_activity', 'Fp2_mobility', 'Fp2_complexity', 'F4_activity', 'F4_mobility', 'F4_complexity', 'C4_activity', 'C4_mobility', 'C4_complexity', 'P4_activity', 'P4_mobility', 'P4_complexity', 'F8_activity', 'F8_mobility', 'F8_complexity', 'T4_activity', 'T4_mobility', 'T4_complexity', 'T6_activity', 'T6_mobility', 'T6_complexity', 'O2_activity', 'O2_mobility', 'O2_complexity', 'EKG_activity', 'EKG_mobility', 'EKG_complexity', 'generalized']


In [9]:
train_data = pd.DataFrame(amc_matrix, columns=columns)
display(train_data.head())
target = df['seizure_vote']
eps = 0.0001
clamp = np.array([min(1 - eps, max(eps, trgt)) for trgt in target])
train_data['Target'] = np.log(clamp / (1 - clamp))
train_data = train_data.dropna()
train_data = train_data.drop(['generalized'],axis=1)
train_data.describe()

,Fp1_activity,Fp1_mobility,Fp1_complexity,F3_activity,F3_mobility,F3_complexity,C3_activity,C3_mobility,C3_complexity,P3_activity,...,T6_activity,T6_mobility,T6_complexity,O2_activity,O2_mobility,O2_complexity,EKG_activity,EKG_mobility,EKG_complexity,generalized
0,665.293601,16.032188,1.675398,520.038523,18.349598,1.521175,390.083790,18.872775,1.483272,352.003469,...,501.156187,15.752103,1.777846,238.483457,13.458633,2.109859,15532.655382,32.320484,1.082008,11.505031
1,556.161667,18.132884,1.556165,518.136416,19.605551,1.465214,372.762332,20.034656,1.437827,392.201374,...,524.444110,19.284937,1.556305,265.271249,15.291667,1.956083,16041.473549,31.998661,1.087207,6.316707
2,677.432404,15.350324,1.798947,644.921569,17.021152,1.643488,425.840380,18.367061,1.535985,485.410011,...,647.486065,18.050896,1.657863,297.314741,14.858346,1.996606,16807.800383,31.989682,1.087654,5.926386
3,372.277420,19.506661,1.388109,338.154312,21.463807,1.377466,239.085570,23.633342,1.304974,265.549726,...,537.684322,21.766047,1.486522,323.389891,15.785013,1.966673,15062.097664,31.850900,1.086436,4.501923
4,479.726714,17.414981,1.648074,357.822601,20.575406,1.428399,310.973712,21.453501,1.402090,257.080801,...,963.444987,18.358732,1.770393,314.735952,17.857547,1.786694,15210.044617,31.720912,1.088588,4.916056


,Fp1_activity,Fp1_mobility,Fp1_complexity,F3_activity,F3_mobility,F3_complexity,C3_activity,C3_mobility,C3_complexity,P3_activity,...,T6_activity,T6_mobility,T6_complexity,O2_activity,O2_mobility,O2_complexity,EKG_activity,EKG_mobility,EKG_complexity,Target
count,1.062670e+05,106267.000000,106267.000000,1.062670e+05,106267.000000,106267.000000,1.062670e+05,106267.000000,106267.000000,1.062670e+05,...,1.062670e+05,106267.000000,106267.000000,1.062670e+05,106267.000000,106267.000000,1.062670e+05,106267.000000,106267.000000,106267.000000
mean,3.671043e+05,11.272793,2.787274,2.191425e+05,12.371481,2.600443,2.409786e+05,12.699680,2.623710,2.429978e+05,...,1.954208e+05,12.761131,2.747281,2.829153e+05,12.403829,2.848648,1.483666e+08,18.179527,2.019811,-4.950708
std,7.484579e+06,5.025840,1.961052,5.398418e+06,5.162411,1.878940,5.702787e+06,5.287492,2.005594,5.647362e+06,...,4.716815e+06,5.459030,2.233552,6.625059e+06,5.417394,2.365799,3.569068e+09,7.418314,1.594928,7.003645
min,3.734128e-03,0.287728,0.898326,3.734128e-03,0.376104,1.003815,3.734128e-03,0.348235,0.999540,3.734128e-03,...,3.734128e-03,0.343358,0.852522,3.734128e-03,0.384182,0.980129,3.734128e-03,0.394600,0.863780,-9.210240
25%,6.166360e+02,7.923282,1.791654,4.607481e+02,8.968045,1.715370,3.573589e+02,9.148157,1.714979,3.454494e+02,...,2.562665e+02,9.182935,1.725487,2.697047e+02,8.800797,1.755457,1.644726e+02,13.199608,1.334262,-9.210240
50%,1.402344e+03,10.634083,2.259698,1.063081e+03,11.913824,2.106776,7.963957e+02,12.279902,2.109468,7.381424e+02,...,6.081972e+02,12.431893,2.118072,6.092442e+02,12.197068,2.160924,2.540505e+03,16.985136,1.695193,-9.210240
75%,3.827696e+03,14.006697,3.029664,2.636126e+03,15.319450,2.758089,2.117809e+03,15.886015,2.754236,1.984496e+03,...,1.743501e+03,16.200959,2.814494,1.815936e+03,15.755370,2.915967,1.160521e+04,24.595897,1.988830,-1.386294
max,3.656521e+08,37.454902,55.309563,3.497580e+08,37.454902,56.454374,3.504261e+08,37.454902,76.721602,3.483625e+08,...,3.447537e+08,37.454902,58.587997,3.831805e+08,37.454902,67.365412,1.430061e+11,37.955465,47.422840,9.210240


In [10]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(train_data.drop(['Target'],axis=1), train_data['Target'])
score = model.score(train_data.drop(['Target'],axis=1), train_data['Target'])
print(f'Score: {score}')

Score: 0.07044894884576014
